In [1]:
import numpy as np
from simtk import unit as unit
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
def Hyd_file_parser(filename):
    init=False
    contacts_traj = []
    contacts_dict = {}
    n_contacts = 0
    with open(filename, 'r') as fff:
        for line in fff.readlines():
            if line.startswith('==='):
                if init:
                    contacts_traj.append(contacts_frame)
                contacts_frame = []
                init=True
            if line.startswith('  to'):
                words = line.split()
                resid = words[2]+'-'+words[4]+'/'+words[3]
                n_interactions = int(words[6])
                strength = float(words[-1])
                contact_label = 'GBC-985/A||'+resid
                try:
                    contact_index = contacts_dict[contact_label]
                except:
                    contact_index = n_contacts
                    contacts_dict[contact_label] = contact_index
                    n_contacts += 1
                contacts_frame.append([contact_index, n_interactions, strength])
    return contacts_traj, contacts_dict

In [3]:
def HB_file_parser(filename):
    init=False
    contacts_traj = []
    contacts_dict = {}
    n_contacts = 0
    with open(filename, 'r') as fff:
        for line in fff.readlines():
            if line.startswith('==='):
                if init:
                    contacts_traj.append(contacts_frame)
                contacts_frame = []
                init=True
            if line.startswith('Atom'):
                words = line.split()
                donor = words[1]+'/'+words[2]+'-'+words[3]+'/'+words[4]
                acceptor = words[9]+'/'+words[10]+'-'+words[11]+'/'+words[12][0]
                contact_label = donor+'||'+acceptor
                try:
                    contact_index = contacts_dict[contact_label]
                except:
                    contact_index = n_contacts
                    contacts_dict[contact_label] = contact_index
                    n_contacts += 1
                distance = float(words[17])*unit.angstroms
                energy = float(words[22])*unit.kilojoules_per_mole
                contacts_frame.append([contact_index, distance, energy])
    return contacts_traj, contacts_dict

## With out thresholds

In [4]:
file_HB="contactos/HB_pose1.txt"
file_Hyd="contactos/Hyd_pose1.txt"

traj_HB, dict_HB = HB_file_parser(file_HB)
traj_Hyd, dict_Hyd = Hyd_file_parser(file_Hyd)

mss_traj = []

In [5]:
for frame_HB, frame_Hyd in zip(traj_HB, traj_Hyd):
    contact_indices_HB = np.sort([ii[0] for ii in frame_HB])
    contact_indices_Hyd = np.sort([ii[0] for ii in frame_Hyd])
    mss_HB = ','.join([str(ii) for ii in contact_indices_HB])
    mss_Hyd = ','.join([str(ii) for ii in contact_indices_Hyd])
    mss = mss_HB+'-'+mss_Hyd
    mss_traj.append(mss)

In [8]:
fff = open('dict_HB_pose1.txt','w')
for key, value in dict_HB.items():
    fff.write("{}\t{}\n".format(value,key))
fff.close()

fff = open('dict_Hyd_pose1.txt','w')
for key, value in dict_HB.items():
    fff.write("{}\t{}\n".format(value,key))
fff.close()

fff = open('traj_pose1.txt','w')
for mss in mss_traj:
    fff.write("{}\n".format(mss))
fff.close()